In [1]:
from __future__ import print_function
import os
from skimage import io, transform
import argparse
import torch
import torchvision
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
from torchvision import datasets, transforms
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

In [2]:
batch_size = 100   # input batch size for training
epochs = 5       # number of epochs to train
lr = 0.01
num_inputs_1 = 3072
num_outputs_1 = 1024
num_outputs_2 = 256
num_outputs_3 = 17



In [8]:
class data(Dataset):

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.file = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.file)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,
                                self.file.iloc[idx, 0])
        image = Image.open(img_name+'.jpg')
        image = image.convert('RGB')
#         image = io.imread()
#         image = Image.fromarray(image.numpy(), mode='L')
        if self.transform is not None:
            image = self.transform(image)
        labels = self.file.iloc[idx, 1].astype('int')

        

        return (image,labels)


In [9]:
data_transform = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor()
])
train_dataset = data(csv_file='CSCI-GA.3033-023/kaggleamazon/train.csv', root_dir='CSCI-GA.3033-023/kaggleamazon/train-jpg/',transform = data_transform)
print(train_dataset[0])
train_loader = DataLoader(train_dataset,batch_size, num_workers=1)
test_dataset = data(csv_file='CSCI-GA.3033-023/kaggleamazon/test.csv', root_dir='CSCI-GA.3033-023/kaggleamazon/train-jpg/',transform = data_transform)
test_loader = DataLoader(test_dataset, batch_size,  num_workers=1)






(
(0 ,.,.) = 
  0.3725  0.3647  0.3608  ...   0.3569  0.3569  0.3529
  0.3647  0.3569  0.3490  ...   0.3529  0.3490  0.3412
  0.3686  0.3608  0.3451  ...   0.3569  0.3529  0.3412
           ...             ⋱             ...          
  0.3569  0.3647  0.3647  ...   0.3529  0.3569  0.3608
  0.3608  0.3647  0.3647  ...   0.3608  0.3608  0.3569
  0.3686  0.3608  0.3647  ...   0.3529  0.3569  0.3451

(1 ,.,.) = 
  0.4314  0.4275  0.4235  ...   0.4275  0.4235  0.4235
  0.4235  0.4235  0.4196  ...   0.4275  0.4196  0.4196
  0.4314  0.4275  0.4196  ...   0.4235  0.4235  0.4157
           ...             ⋱             ...          
  0.4235  0.4275  0.4353  ...   0.4275  0.4275  0.4275
  0.4314  0.4275  0.4392  ...   0.4235  0.4275  0.4275
  0.4392  0.4314  0.4353  ...   0.4235  0.4275  0.4235

(2 ,.,.) = 
  0.4039  0.4039  0.4039  ...   0.3961  0.4000  0.4000
  0.4039  0.4039  0.4000  ...   0.4000  0.3961  0.3922
  0.4078  0.4039  0.4000  ...   0.4000  0.4000  0.3961
           ...           

In [10]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.linear1 = nn.Linear(num_inputs_1, num_outputs_1)
        self.linear2 = nn.Linear(num_outputs_1, num_outputs_2)
        self.linear3 = nn.Linear(num_outputs_2, num_outputs_3)

    def forward(self, input):
        input = input.view(-1, num_inputs_1) # reshape input to batch x num_inputs
        z = F.relu(self.linear1(input))
        z = F.relu(self.linear2(z))
        output = F.log_softmax(self.linear3(z))
        return output
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum = 0.9)


In [11]:
def train(epoch, break_val):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = net(data)
        loss = criterion(output, target)
        print(batch_idx, loss.data[0])
        loss.backward()
        optimizer.step()
        if batch_idx==break_val:
            return
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))



In [12]:
for epoch in range(1, epochs + 1):
    train(epoch,500)

0 2.8452486991882324
Train Epoch: 1 [0/20000 (0%)]	Loss: 2.845249
1 2.81648588180542
2 2.7909789085388184
3 2.739372968673706
4 2.7126548290252686
5 2.6436591148376465
6 2.5455737113952637
7 2.4946136474609375
8 2.3641555309295654
9 2.249326705932617
10 2.06950306892395
11 1.9261102676391602
12 1.7968647480010986


Process Process-5:
Traceback (most recent call last):
  File "/Users/srinidhigoud/anaconda/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/srinidhigoud/anaconda/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 40, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/Users/srinidhigoud/anaconda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/srinidhigoud/anaconda/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 40, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "<ipython-input-8-21dbd6681e8b>", line 22, in __getitem__
    image = image.convert('RGB')
  File "/Users/srinidhigoud/anaconda/lib/python3.6/site-packages/PIL/Image.py", line 877, in convert
    self.load()
  File "/Users/srinidhigoud/anaconda/lib/python3.6/site-packages/PIL/ImageFile.py", line 236, in load


13 1.9774214029312134


KeyboardInterrupt: 